<a href="https://colab.research.google.com/github/plaban1981/indic_language/blob/main/Indic_language_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 49.0 MB/s eta 0:00:00


In [4]:
!pip install -q sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.9 MB/s eta 0:00:00


In [10]:
import shutil
shutil.move("/content/model_files/model_configs","/content")

'/content/model_configs'

In [22]:
shutil.move("/content/model_configs","/content/model_files/en-indic")

'/content/model_files/en-indic/model_configs'

In [1]:
# clone the repo for running evaluation
!git clone https://github.com/AI4Bharat/indicTrans.git
%cd indicTrans
# clone requirements repositories
!git clone https://github.com/anoopkunchukuttan/indic_nlp_library.git
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!git clone https://github.com/rsennrich/subword-nmt.git
%cd ..

Cloning into 'indicTrans'...
remote: Enumerating objects: 694, done.
remote: Counting objects: 100% (397/397), done.
remote: Compressing objects: 100% (201/201), done.
remote: Total 694 (delta 273), reused 272 (delta 194), pack-reused 297
Receiving objects: 100% (694/694), 2.65 MiB | 24.00 MiB/s, done.
Resolving deltas: 100% (400/400), done.
/content/indicTrans
Cloning into 'indic_nlp_library'...
remote: Enumerating objects: 1325, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 1325 (delta 91), reused 82 (delta 82), pack-reused 1218
Receiving objects: 100% (1325/1325), 9.55 MiB | 14.05 MiB/s, done.
Resolving deltas: 100% (701/701), done.
Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 139 (delta 2), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (139/139), 149.77 MiB |

In [8]:
rm -rf "/content/indicTrans"

In [2]:
%%writefile requirements.txt
sacremoses
pandas
mock
sacrebleu
pyarrow
indic-nlp-library
mosestokenizer
subword-nmt
numpy
tensorboardX
git+https://github.com/pytorch/fairseq.git

Writing requirements.txt


In [4]:
!nvidia-smi

Mon Mar 13 14:32:10 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0    24W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/pytorch/fairseq.git to /tmp/pip-req-build-q4vuzuwu
  Running command git clone --filter=blob:none --quiet https://github.com/pytorch/fairseq.git /tmp/pip-req-build-q4vuzuwu
  Resolved https://github.com/pytorch/fairseq.git to commit 0338cdc3094ca7d29ff4d36d64791f7b4e4b5e6e
  Running command git submodule update --init --recursive -q
  From https://github.com/ngoyal2707/Megatron-LM
   * branch            adb23324c222aad0aad89308e70302d996a5eaeb -> FETCH_HEAD
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 37.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 KB 4.6 MB/s eta 

## download the indictrans model


#### downloading the indic-en model

!wget https://ai4b-public-nlu-nlg.objectstore.e2enetworks.net/indic2en.zip

!unzip indic2en.zip

#### downloading the en-indic model
!wget https://ai4b-public-nlu-nlg.objectstore.e2enetworks.net/en2indic.zip

!unzip en2indic.zip

#### downloading the indic-indic model
!wget https://ai4b-public-nlu-nlg.objectstore.e2enetworks.net/m2m.zip

!unzip m2m.zip

## English to Indic Languages

In [6]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [5]:
!mkdir model_files

In [8]:
import shutil
shutil.move("/content/indicTrans/model_configs","/content/drive/MyDrive/indic/model_files/en-indic")

'/content/drive/MyDrive/indic/model_files/en-indic/model_configs'

In [4]:
!wget "https://ai4b-public-nlu-nlg.objectstore.e2enetworks.net/en2indic.zip"

--2023-03-13 13:19:07--  https://ai4b-public-nlu-nlg.objectstore.e2enetworks.net/en2indic.zip
Resolving ai4b-public-nlu-nlg.objectstore.e2enetworks.net (ai4b-public-nlu-nlg.objectstore.e2enetworks.net)... 164.52.206.154, 101.53.136.19, 101.53.136.18, ...
Connecting to ai4b-public-nlu-nlg.objectstore.e2enetworks.net (ai4b-public-nlu-nlg.objectstore.e2enetworks.net)|164.52.206.154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4811880516 (4.5G) [application/zip]
Saving to: ‘en2indic.zip’

en2indic.zip        100%[===================>]   4.48G  12.4MB/s    in 6m 46s  

2023-03-13 13:25:55 (11.3 MB/s) - ‘en2indic.zip’ saved [4811880516/4811880516]



In [5]:
!unzip "/content/en2indic.zip" -d "/content/model_files"

Archive:  /content/en2indic.zip
   creating: /content/model_files/en-indic/
   creating: /content/model_files/en-indic/vocab/
  inflating: /content/model_files/en-indic/vocab/bpe_codes.32k.SRC  
  inflating: /content/model_files/en-indic/vocab/vocab.SRC  
  inflating: /content/model_files/en-indic/vocab/vocab.TGT  
  inflating: /content/model_files/en-indic/vocab/bpe_codes.32k.TGT  
   creating: /content/model_files/en-indic/final_bin/
  inflating: /content/model_files/en-indic/final_bin/preprocess.log  
  inflating: /content/model_files/en-indic/final_bin/dict.TGT.txt  
  inflating: /content/model_files/en-indic/final_bin/test.SRC-TGT.SRC.idx  
  inflating: /content/model_files/en-indic/final_bin/test.SRC-TGT.TGT.idx  
  inflating: /content/model_files/en-indic/final_bin/train.SRC-TGT.SRC.idx  
  inflating: /content/model_files/en-indic/final_bin/dict.SRC.txt  
  inflating: /content/model_files/en-indic/final_bin/valid.SRC-TGT.TGT.idx  
  inflating: /content/model_files/en-indic/final

In [48]:
!wget https://ai4b-public-nlu-nlg.objectstore.e2enetworks.net/indic2en.zip

--2023-03-13 12:11:38--  https://ai4b-public-nlu-nlg.objectstore.e2enetworks.net/indic2en.zip
Resolving ai4b-public-nlu-nlg.objectstore.e2enetworks.net (ai4b-public-nlu-nlg.objectstore.e2enetworks.net)... 164.52.206.155, 164.52.210.57, 101.53.152.30, ...
Connecting to ai4b-public-nlu-nlg.objectstore.e2enetworks.net (ai4b-public-nlu-nlg.objectstore.e2enetworks.net)|164.52.206.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4759117228 (4.4G) [application/zip]
Saving to: ‘indic2en.zip.1’

indic2en.zip.1      100%[===================>]   4.43G  13.3MB/s    in 5m 56s  

2023-03-13 12:17:36 (12.7 MB/s) - ‘indic2en.zip.1’ saved [4759117228/4759117228]



In [56]:
!unzip indic2en.zip -d /content/model_files

Archive:  indic2en.zip
   creating: /content/model_files/indic-en/
   creating: /content/model_files/indic-en/vocab/
  inflating: /content/model_files/indic-en/vocab/bpe_codes.32k.SRC  
  inflating: /content/model_files/indic-en/vocab/vocab.SRC  
  inflating: /content/model_files/indic-en/vocab/vocab.TGT  
  inflating: /content/model_files/indic-en/vocab/bpe_codes.32k.TGT  
   creating: /content/model_files/indic-en/final_bin/
  inflating: /content/model_files/indic-en/final_bin/preprocess.log  
  inflating: /content/model_files/indic-en/final_bin/dict.TGT.txt  
  inflating: /content/model_files/indic-en/final_bin/test.SRC-TGT.SRC.idx  
  inflating: /content/model_files/indic-en/final_bin/test.SRC-TGT.TGT.idx  
  inflating: /content/model_files/indic-en/final_bin/train.SRC-TGT.SRC.idx  
  inflating: /content/model_files/indic-en/final_bin/dict.SRC.txt  
  inflating: /content/model_files/indic-en/final_bin/valid.SRC-TGT.TGT.idx  
  inflating: /content/model_files/indic-en/final_bin/test

In [49]:
!unzip /content/en-indic.zip -d /content/model_files

Archive:  /content/en-indic.zip
   creating: /content/model_files/en-indic/
   creating: /content/model_files/en-indic/vocab/
  inflating: /content/model_files/en-indic/vocab/bpe_codes.32k.SRC  
  inflating: /content/model_files/en-indic/vocab/vocab.SRC  
  inflating: /content/model_files/en-indic/vocab/vocab.TGT  
  inflating: /content/model_files/en-indic/vocab/bpe_codes.32k.TGT  
   creating: /content/model_files/en-indic/final_bin/
  inflating: /content/model_files/en-indic/final_bin/preprocess.log  
  inflating: /content/model_files/en-indic/final_bin/dict.TGT.txt  
  inflating: /content/model_files/en-indic/final_bin/test.SRC-TGT.SRC.idx  
  inflating: /content/model_files/en-indic/final_bin/test.SRC-TGT.TGT.idx  
  inflating: /content/model_files/en-indic/final_bin/train.SRC-TGT.SRC.idx  
  inflating: /content/model_files/en-indic/final_bin/dict.SRC.txt  
  inflating: /content/model_files/en-indic/final_bin/valid.SRC-TGT.TGT.idx  
  inflating: /content/model_files/en-indic/final

In [9]:
# clone the repo for running evaluation
!git clone https://github.com/AI4Bharat/indicTrans.git
%cd indicTrans
# clone requirements repositories
!git clone https://github.com/anoopkunchukuttan/indic_nlp_library.git
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!git clone https://github.com/rsennrich/subword-nmt.git
%cd ..

Cloning into 'indicTrans'...
remote: Enumerating objects: 694, done.
remote: Counting objects: 100% (397/397), done.
remote: Compressing objects: 100% (201/201), done.
remote: Total 694 (delta 273), reused 272 (delta 194), pack-reused 297
Receiving objects: 100% (694/694), 2.65 MiB | 14.27 MiB/s, done.
Resolving deltas: 100% (400/400), done.
/content/indicTrans
Cloning into 'indic_nlp_library'...
remote: Enumerating objects: 1325, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 1325 (delta 91), reused 82 (delta 82), pack-reused 1218
Receiving objects: 100% (1325/1325), 9.55 MiB | 9.98 MiB/s, done.
Resolving deltas: 100% (701/701), done.
Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 139 (delta 2), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (139/139), 149.77 MiB | 

In [1]:
"""
Translate raw text with a trained model. Batches data on-the-fly.
"""

import ast
from collections import namedtuple

import torch
from fairseq import checkpoint_utils, options, tasks, utils
from fairseq.dataclass.utils import convert_namespace_to_omegaconf
from fairseq.token_generation_constraints import pack_constraints, unpack_constraints
from fairseq_cli.generate import get_symbols_to_strip_from_output

import codecs


Batch = namedtuple("Batch", "ids src_tokens src_lengths constraints")
Translation = namedtuple("Translation", "src_str hypos pos_scores alignments")


def make_batches(
    lines, cfg, task, max_positions, encode_fn, constrainted_decoding=False
):
    def encode_fn_target(x):
        return encode_fn(x)

    if constrainted_decoding:
        # Strip (tab-delimited) contraints, if present, from input lines,
        # store them in batch_constraints
        batch_constraints = [list() for _ in lines]
        for i, line in enumerate(lines):
            if "\t" in line:
                lines[i], *batch_constraints[i] = line.split("\t")

        # Convert each List[str] to List[Tensor]
        for i, constraint_list in enumerate(batch_constraints):
            batch_constraints[i] = [
                task.target_dictionary.encode_line(
                    encode_fn_target(constraint),
                    append_eos=False,
                    add_if_not_exist=False,
                )
                for constraint in constraint_list
            ]

    if constrainted_decoding:
        constraints_tensor = pack_constraints(batch_constraints)
    else:
        constraints_tensor = None

    tokens, lengths = task.get_interactive_tokens_and_lengths(lines, encode_fn)

    itr = task.get_batch_iterator(
        dataset=task.build_dataset_for_inference(
            tokens, lengths, constraints=constraints_tensor
        ),
        max_tokens=cfg.dataset.max_tokens,
        max_sentences=cfg.dataset.batch_size,
        max_positions=max_positions,
        ignore_invalid_inputs=cfg.dataset.skip_invalid_size_inputs_valid_test,
    ).next_epoch_itr(shuffle=False)
    for batch in itr:
        ids = batch["id"]
        src_tokens = batch["net_input"]["src_tokens"]
        src_lengths = batch["net_input"]["src_lengths"]
        constraints = batch.get("constraints", None)

        yield Batch(
            ids=ids,
            src_tokens=src_tokens,
            src_lengths=src_lengths,
            constraints=constraints,
        )


class Translator:
    def __init__(
        self, model_path,data_dir, checkpoint_path, batch_size=25, constrained_decoding=False
    ):

        self.constrained_decoding = constrained_decoding
        self.parser = options.get_generation_parser(interactive=True)
        self.model_path = model_path
        # buffer_size is currently not used but we just initialize it to batch
        # size + 1 to avoid any assertion errors.
        if self.constrained_decoding:
            self.parser.set_defaults(
                path=checkpoint_path,
                remove_bpe="subword_nmt",
                num_workers=-1,
                constraints="ordered",
                batch_size=batch_size,
                buffer_size=batch_size + 1,
            )
        else:
            self.parser.set_defaults(
                path=checkpoint_path,
                remove_bpe="subword_nmt",
                num_workers=-1,
                batch_size=batch_size,
                buffer_size=batch_size + 1,
            )
        args = options.parse_args_and_arch(self.parser, input_args=[data_dir])
        # we are explictly setting src_lang and tgt_lang here
        # generally the data_dir we pass contains {split}-{src_lang}-{tgt_lang}.*.idx files from
        # which fairseq infers the src and tgt langs(if these are not passed). In deployment we dont
        # use any idx files and only store the SRC and TGT dictionaries.
        args.source_lang = "SRC"
        args.target_lang = "TGT"
        # since we are truncating sentences to max_seq_len in engine, we can set it to False here
        args.skip_invalid_size_inputs_valid_test = False

        # we have custom architechtures in this folder and we will let fairseq
        # import this
        model_file_path = os.path.join(self.model_path,"en-indic","model_configs")
        args.user_dir = "model_configs"
        self.cfg = convert_namespace_to_omegaconf(args)

        utils.import_user_module(self.cfg.common)

        if self.cfg.interactive.buffer_size < 1:
            self.cfg.interactive.buffer_size = 1
        if self.cfg.dataset.max_tokens is None and self.cfg.dataset.batch_size is None:
            self.cfg.dataset.batch_size = 1

        assert (
            not self.cfg.generation.sampling
            or self.cfg.generation.nbest == self.cfg.generation.beam
        ), "--sampling requires --nbest to be equal to --beam"
        assert (
            not self.cfg.dataset.batch_size
            or self.cfg.dataset.batch_size <= self.cfg.interactive.buffer_size
        ), "--batch-size cannot be larger than --buffer-size"

        # Fix seed for stochastic decoding
        # if self.cfg.common.seed is not None and not self.cfg.generation.no_seed_provided:
        #     np.random.seed(self.cfg.common.seed)
        #     utils.set_torch_seed(self.cfg.common.seed)

        # if not self.constrained_decoding:
        #     self.use_cuda = torch.cuda.is_available() and not self.cfg.common.cpu
        # else:
        #     self.use_cuda = False

        #self.use_cuda = torch.cuda.is_available() and not self.cfg.common.cpu
        self.use_cuda = torch.cuda.is_available()
        # Setup task, e.g., translation
        self.task = tasks.setup_task(self.cfg.task)

        # Load ensemble
        overrides = ast.literal_eval(self.cfg.common_eval.model_overrides)
        self.models, self._model_args = checkpoint_utils.load_model_ensemble(
            utils.split_paths(self.cfg.common_eval.path),
            arg_overrides=overrides,
            task=self.task,
            suffix=self.cfg.checkpoint.checkpoint_suffix,
            strict=(self.cfg.checkpoint.checkpoint_shard_count == 1),
            num_shards=self.cfg.checkpoint.checkpoint_shard_count,
        )

        # Set dictionaries
        self.src_dict = self.task.source_dictionary
        self.tgt_dict = self.task.target_dictionary

        # Optimize ensemble for generation
        for model in self.models:
            if model is None:
                continue
            if self.cfg.common.fp16:
                model.half()
            if (
                self.use_cuda
                and not self.cfg.distributed_training.pipeline_model_parallel
            ):
                model.cuda()
            model.prepare_for_inference_(self.cfg)

        # Initialize generator
        self.generator = self.task.build_generator(self.models, self.cfg.generation)

        # Handle tokenization and BPE
        self.tokenizer = self.task.build_tokenizer(self.cfg.tokenizer)
        self.bpe = self.task.build_bpe(self.cfg.bpe)

        # Load alignment dictionary for unknown word replacement
        # (None if no unknown word replacement, empty if no path to align dictionary)
        self.align_dict = utils.load_align_dict(self.cfg.generation.replace_unk)

        self.max_positions = utils.resolve_max_positions(
            self.task.max_positions(), *[model.max_positions() for model in self.models]
        )

    def encode_fn(self, x):
        if self.tokenizer is not None:
            x = self.tokenizer.encode(x)
        if self.bpe is not None:
            x = self.bpe.encode(x)
        return x

    def decode_fn(self, x):
        if self.bpe is not None:
            x = self.bpe.decode(x)
        if self.tokenizer is not None:
            x = self.tokenizer.decode(x)
        return x

    def translate(self, inputs, constraints=None):
        if self.constrained_decoding and constraints is None:
            raise ValueError("Constraints cant be None in constrained decoding mode")
        if not self.constrained_decoding and constraints is not None:
            raise ValueError("Cannot pass constraints during normal translation")
        if constraints:
            constrained_decoding = True
            modified_inputs = []
            for _input, constraint in zip(inputs, constraints):
                modified_inputs.append(_input + f"\t{constraint}")
            inputs = modified_inputs
        else:
            constrained_decoding = False

        start_id = 0
        results = []
        final_translations = []
        for batch in make_batches(
            inputs,
            self.cfg,
            self.task,
            self.max_positions,
            self.encode_fn,
            constrained_decoding,
        ):
            bsz = batch.src_tokens.size(0)
            src_tokens = batch.src_tokens
            src_lengths = batch.src_lengths
            constraints = batch.constraints
            if self.use_cuda:
                src_tokens = src_tokens.cuda()
                src_lengths = src_lengths.cuda()
                if constraints is not None:
                    constraints = constraints.cuda()

            sample = {
                "net_input": {
                    "src_tokens": src_tokens,
                    "src_lengths": src_lengths,
                },
            }

            translations = self.task.inference_step(
                self.generator, self.models, sample, constraints=constraints
            )

            list_constraints = [[] for _ in range(bsz)]
            if constrained_decoding:
                list_constraints = [unpack_constraints(c) for c in constraints]
            for i, (id, hypos) in enumerate(zip(batch.ids.tolist(), translations)):
                src_tokens_i = utils.strip_pad(src_tokens[i], self.tgt_dict.pad())
                constraints = list_constraints[i]
                results.append(
                    (
                        start_id + id,
                        src_tokens_i,
                        hypos,
                        {
                            "constraints": constraints,
                        },
                    )
                )

        # sort output to match input order
        for id_, src_tokens, hypos, _ in sorted(results, key=lambda x: x[0]):
            src_str = ""
            if self.src_dict is not None:
                src_str = self.src_dict.string(
                    src_tokens, self.cfg.common_eval.post_process
                )

            # Process top predictions
            for hypo in hypos[: min(len(hypos), self.cfg.generation.nbest)]:
                hypo_tokens, hypo_str, alignment = utils.post_process_prediction(
                    hypo_tokens=hypo["tokens"].int().cpu(),
                    src_str=src_str,
                    alignment=hypo["alignment"],
                    align_dict=self.align_dict,
                    tgt_dict=self.tgt_dict,
                    remove_bpe="subword_nmt",
                    extra_symbols_to_ignore=get_symbols_to_strip_from_output(
                        self.generator
                    ),
                )
                detok_hypo_str = self.decode_fn(hypo_str)
                final_translations.append(detok_hypo_str)
        return final_translations

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [19]:
from os import truncate
from sacremoses import MosesPunctNormalizer
from sacremoses import MosesTokenizer
from sacremoses import MosesDetokenizer
from subword_nmt.apply_bpe import BPE, read_vocabulary
import codecs
from tqdm import tqdm
from indicnlp.tokenize import indic_tokenize
from indicnlp.tokenize import indic_detokenize
from indicnlp.normalize import indic_normalize
from indicnlp.transliterate import unicode_transliterate
from mosestokenizer import MosesSentenceSplitter
from indicnlp.tokenize import sentence_tokenize

#from inference.custom_interactive import Translator
#Translator = Translator()

INDIC = ["as", "bn", "gu", "hi", "kn", "ml", "mr", "or", "pa", "ta", "te"]


def split_sentences(paragraph, language):
    if language == "en":
        with MosesSentenceSplitter(language) as splitter:
            return splitter([paragraph])
    elif language in INDIC:
        return sentence_tokenize.sentence_split(paragraph, lang=language)


def add_token(sent, tag_infos):
    """add special tokens specified by tag_infos to each element in list
    tag_infos: list of tuples (tag_type,tag)
    each tag_info results in a token of the form: __{tag_type}__{tag}__
    """

    tokens = []
    for tag_type, tag in tag_infos:
        token = "__" + tag_type + "__" + tag + "__"
        tokens.append(token)

    return " ".join(tokens) + " " + sent


def apply_lang_tags(sents, src_lang, tgt_lang):
    tagged_sents = []
    for sent in sents:
        tagged_sent = add_token(sent.strip(), [("src", src_lang), ("tgt", tgt_lang)])
        tagged_sents.append(tagged_sent)
    return tagged_sents


def truncate_long_sentences(sents):

    MAX_SEQ_LEN = 200
    new_sents = []

    for sent in sents:
        words = sent.split()
        num_words = len(words)
        if num_words > MAX_SEQ_LEN:
            print_str = " ".join(words[:5]) + " .... " + " ".join(words[-5:])
            sent = " ".join(words[:MAX_SEQ_LEN])
            print(
                f"WARNING: Sentence {print_str} truncated to 200 tokens as it exceeds maximum length limit"
            )

        new_sents.append(sent)
    return new_sents


class Model:
    def __init__(self, expdir,model_path):
        self.expdir = expdir
        self.model_path = model_path
        self.en_tok = MosesTokenizer(lang="en")
        self.en_normalizer = MosesPunctNormalizer()
        self.en_detok = MosesDetokenizer(lang="en")
        self.xliterator = unicode_transliterate.UnicodeIndicTransliterator()
        print("Initializing vocab and bpe")
        self.vocabulary = read_vocabulary(
            codecs.open(f"{expdir}/vocab/vocab.SRC", encoding="utf-8"), 5
        )
        self.bpe = BPE(
            codecs.open(f"{expdir}/vocab/bpe_codes.32k.SRC", encoding="utf-8"),
            -1,
            "@@",
            self.vocabulary,
            None,
        )

        print("Initializing model for translation")
        # initialize the model
        self.translator = Translator(
            model_path,f"{expdir}/final_bin", f"{expdir}/model/checkpoint_best.pt", batch_size=100
        )

    # translate a batch of sentences from src_lang to tgt_lang
    def batch_translate(self, batch, src_lang, tgt_lang):

        assert isinstance(batch, list)
        preprocessed_sents = self.preprocess(batch, lang=src_lang)
        bpe_sents = self.apply_bpe(preprocessed_sents)
        tagged_sents = apply_lang_tags(bpe_sents, src_lang, tgt_lang)
        tagged_sents = truncate_long_sentences(tagged_sents)

        translations = self.translator.translate(tagged_sents)
        postprocessed_sents = self.postprocess(translations, tgt_lang)

        return postprocessed_sents

    # translate a paragraph from src_lang to tgt_lang
    def translate_paragraph(self, paragraph, src_lang, tgt_lang):

        assert isinstance(paragraph, str)
        sents = split_sentences(paragraph, src_lang)

        postprocessed_sents = self.batch_translate(sents, src_lang, tgt_lang)

        translated_paragraph = " ".join(postprocessed_sents)

        return translated_paragraph

    def preprocess_sent(self, sent, normalizer, lang):
        if lang == "en":
            return " ".join(
                self.en_tok.tokenize(
                    self.en_normalizer.normalize(sent.strip()), escape=False
                )
            )
        else:
            # line = indic_detokenize.trivial_detokenize(line.strip(), lang)
            return unicode_transliterate.UnicodeIndicTransliterator.transliterate(
                " ".join(
                    indic_tokenize.trivial_tokenize(
                        normalizer.normalize(sent.strip()), lang
                    )
                ),
                lang,
                "hi",
            ).replace(" ् ", "्")

    def preprocess(self, sents, lang):
        """
        Normalize, tokenize and script convert(for Indic)
        return number of sentences input file
        """

        if lang == "en":

            # processed_sents = Parallel(n_jobs=-1, backend="multiprocessing")(
            #     delayed(preprocess_line)(line, None, lang) for line in tqdm(sents, total=num_lines)
            # )
            processed_sents = [
                self.preprocess_sent(line, None, lang) for line in tqdm(sents)
            ]

        else:
            normfactory = indic_normalize.IndicNormalizerFactory()
            normalizer = normfactory.get_normalizer(lang)

            # processed_sents = Parallel(n_jobs=-1, backend="multiprocessing")(
            #     delayed(preprocess_line)(line, normalizer, lang) for line in tqdm(infile, total=num_lines)
            # )
            processed_sents = [
                self.preprocess_sent(line, normalizer, lang) for line in tqdm(sents)
            ]

        return processed_sents

    def postprocess(self, sents, lang, common_lang="hi"):
        """
        parse fairseq interactive output, convert script back to native Indic script (in case of Indic languages) and detokenize.
        infname: fairseq log file
        outfname: output file of translation (sentences not translated contain the dummy string 'DUMMY_OUTPUT'
        input_size: expected number of output sentences
        lang: language
        """
        postprocessed_sents = []

        if lang == "en":
            for sent in sents:
                # outfile.write(en_detok.detokenize(sent.split(" ")) + "\n")
                postprocessed_sents.append(self.en_detok.detokenize(sent.split(" ")))
        else:
            for sent in sents:
                outstr = indic_detokenize.trivial_detokenize(
                    self.xliterator.transliterate(sent, common_lang, lang), lang
                )
                # outfile.write(outstr + "\n")
                postprocessed_sents.append(outstr)
        return postprocessed_sents

    def apply_bpe(self, sents):

        return [self.bpe.process_line(sent) for sent in sents]


In [11]:
%cd indicTrans

/content/indicTrans


In [40]:
from fairseq import checkpoint_utils, distributed_utils, options, tasks, utils
#from indicTrans.inference.engine import Model
#from inference.engine import Model
#
indic2en_model = Model(expdir='/content/model_files/en-indic')
#
INDIC = {"Assamese": "as", "Bengali": "bn", "Gujarati": "gu", "Hindi": "hi","Kannada": "kn","Malayalam": "ml", "Marathi": "mr", "Odia": "or","Punjabi": "pa","Tamil": "ta", "Telugu" : "te"}
#
languages = list(INDIC.keys())
#

def translate(text, lang):
  return indic2en_model.translate_paragraph(text, 'en', INDIC[lang])

def translate_batch(text, lang):
  return indic2en_model.batch_translate(text, 'en', INDIC[lang])


Initializing vocab and bpe
Initializing model for translation


In [13]:
input_text = ["Elon Musk sells $8.5 billion in Tesla stock", 
              "I'm a professional academic and research writer.",
              "Get a job in US and work in Germany"]

In [14]:
for text in input_text:
  print(translate(text, 'Tamil'))

100%|██████████| 1/1 [00:00<00:00,  1.26it/s]


டெஸ்லா நிறுவனத்தில் 8.5 பில்லியன் டாலர் பங்குகளை விற்றார் எலான் மஸ்க்


100%|██████████| 1/1 [00:00<00:00, 761.08it/s]


நான் ஒரு தொழில்முறை கல்வியாளர் மற்றும் ஆராய்ச்சி எழுத்தாளர்.


100%|██████████| 1/1 [00:00<00:00, 1321.87it/s]


அமெரிக்காவில் வேலை, ஜெர்மனியில் வேலை


In [16]:
translate_batch(input_text, 'Hindi')

100%|██████████| 3/3 [00:00<00:00, 3139.45it/s]


['एलन मस्क ने टेस्ला के 8.5 अरब डॉलर के शेयर बेचे',
 'मैं एक पेशेवर अकादमिक और शोध लेखक हूं।',
 'अमेरिका में नौकरी करो और जर्मनी में काम करो']

In [10]:
%%writefile /content/input.txt
The Sun Temple was built in the 13th century and designed as a gigantic chariot of the Sun God, Surya, with twelve pairs of ornamented wheels pulled by seven horses. Some of the wheels are 3 metres wide. Only six of the seven horse still stand today.The temple fell into disuse after an envoy of Jahangir desecrated the temple in the early 17th century.
According to folklore, there was a diamond in the centre of the idol which reflected the sun rays that passed. In 1627, the then Raja of Khurda took the Sun idol from Konark to the Jagannath temple in Puri. The Sun temple belongs to the Kalingan school of Indian temple architecture. The alignment of the Sun Temple is along the east–west direction. The inner sanctum or vimana used to be surmounted by a tower or shikara but it was razed in the 19th century. The audience hall or jagamohana still stands and comprises majority of the ruins. The roof of the dance hall or natmandir has fallen off. It stands at the eastern end of the ruins on a raised platform.
In 1559, Mukunda Gajapati came to throne in Cuttack. He aligned himself as an ally of Akbar and an enemy of the Sultan of Bengal, Sulaiman Khan Karrani. After a few battles, Odisha finally fell. The fall was also aided by the internal turmoil of the state. In 1568, the Konark temple was damaged by the army of Kalapahad, a general of the Sultan.Kalapahad is also said to be responsible for damages to several other temples during the conquest.;Odia

Writing /content/input.txt


In [7]:
with open("/content/input.txt","r",encoding="utf-8") as rd:
  text = rd.read()
print(text)

The Sun Temple was built in the 13th century and designed as a gigantic chariot of the Sun God, Surya, with twelve pairs of ornamented wheels pulled by seven horses. Some of the wheels are 3 metres wide. Only six of the seven horse still stand today.The temple fell into disuse after an envoy of Jahangir desecrated the temple in the early 17th century.
According to folklore, there was a diamond in the centre of the idol which reflected the sun rays that passed. In 1627, the then Raja of Khurda took the Sun idol from Konark to the Jagannath temple in Puri. The Sun temple belongs to the Kalingan school of Indian temple architecture. The alignment of the Sun Temple is along the east–west direction. The inner sanctum or vimana used to be surmounted by a tower or shikara but it was razed in the 19th century. The audience hall or jagamohana still stands and comprises majority of the ruins. The roof of the dance hall or natmandir has fallen off. It stands at the eastern end of the ruins on a r

In [8]:
input_batch = split_sentences(text, language="en")

NameError: ignored

In [44]:
input_batch

['The Sun Temple was built in the 13th century and designed as a gigantic chariot of the Sun God, Surya, with twelve pairs of ornamented wheels pulled by seven horses.',
 'Some of the wheels are 3 metres wide.',
 'Only six of the seven horse still stand today.The temple fell into disuse after an envoy of Jahangir desecrated the temple in the early 17th century.',
 'According to folklore, there was a diamond in the centre of the idol which reflected the sun rays that passed.',
 'In 1627, the then Raja of Khurda took the Sun idol from Konark to the Jagannath temple in Puri.',
 'The Sun temple belongs to the Kalingan school of Indian temple architecture.',
 'The alignment of the Sun Temple is along the east–west direction.',
 'The inner sanctum or vimana used to be surmounted by a tower or shikara but it was razed in the 19th century.',
 'The audience hall or jagamohana still stands and comprises majority of the ruins.',
 'The roof of the dance hall or natmandir has fallen off.',
 'It sta

In [45]:
lang = 'Odia'
" ".join(indic2en_model.batch_translate(input_batch, 'en', INDIC[lang]))

100%|██████████| 16/16 [00:00<00:00, 1627.83it/s]


'ତ୍ରୟୋଦଶ ଶତାବ୍ଦୀରେ ନିର୍ମିତ ଏହି ସୂର୍ଯ୍ୟମନ୍ଦିର ସୂର୍ଯ୍ୟଦେବଙ୍କ ଏକ ବିଶାଳ ରଥ ଭାବେ ପରିଗଣିତ ହୋଇଥିଲା। କେତେକ ଚକା 3 ମିଟର ଚଉଡ଼ା। ସପ୍ତଦଶ ଶତାବ୍ଦୀର ପ୍ରାରମ୍ଭରେ ଜାହାଙ୍ଗୀରଙ୍କ ଦୂତ ମନ୍ଦିରକୁ ଅପବିତ୍ର କରିବା ପରେ ଏହି ମନ୍ଦିର ନଷ୍ଟ ହୋଇଯାଇଥିଲା। ଲୋକକଥା ଅନୁସାରେ ମୂର୍ତ୍ତିର ମଧ୍ୟଭାଗରେ ଗୋଟିଏ ହୀରା ଥିଲା ଯାହା ସୂର୍ଯ୍ୟକିରଣ ପ୍ରତିଫଳିତ କରୁଥିଲା। 1627 ମସିହାରେ ତତ୍କାଳୀନ ଖୋର୍ଦ୍ଧା ରାଜା ସୂର୍ଯ୍ୟମୂର୍ତ୍ତିକୁ କୋଣାର୍କରୁ ପୁରୀର ଜଗନ୍ନାଥ ମନ୍ଦିରକୁ ନେଇଯାଇଥିଲେ। ଏହି ସୂର୍ଯ୍ୟ ମନ୍ଦିର ଭାରତୀୟ ମନ୍ଦିର ସ୍ଥାପତ୍ୟର କଳିଙ୍ଗନ ବିଦ୍ୟାଳୟର ଅଟେ। ସୂର୍ଯ୍ୟମନ୍ଦିରର ପାର୍ଶ୍ୱସଜ୍ଜା ପୂର୍ବ-ପଶ୍ଚିମ ଦିଗରେ ରହିଛି। ଆଭ୍ୟନ୍ତରୀଣ ଗର୍ଭଗୃହ ବା ବିମାନ ଉପରେ ଏକ ଟାୱାର କିମ୍ବା ଶିକାରା ଥିଲା, କିନ୍ତୁ ଏହା ଉନବିଂଶ ଶତାବ୍ଦୀରେ ଧ୍ୱଂସ ହୋଇଯାଇଥିଲା। ଦର୍ଶକ ହଲ୍ ବା ଜଗମୋହନ ଏବେ ବି ଛିଡ଼ା ହୋଇ ରହିଛି ଏବଂ ସେଥିରେ ଅଧିକାଂଶ ଭଗ୍ନାବଶେଷ ରହିଛି। ନୃତ୍ୟ ଗୃହ କିମ୍ବା ନାଟମନ୍ଦିରର ଛାତ ଉଡିଯାଇଛି। ଏହା ଭଗ୍ନାବଶେଷର ପୂର୍ବ ଦିଗରେ ଏକ ଉପରକୁ ଉଠିଥିବା ପ୍ଲାଟଫର୍ମରେ ରହିଛି। ୧୫୫୯ ମସିହାରେ ମୁକୁନ୍ଦ ଗଜପତି କଟକର ସିଂହାସନ ସମ୍ଭାଳିଥିଲେ। ସେ ନିଜକୁ ଆକବରଙ୍କ ମିତ୍ର ଏବଂ ବଙ୍ଗଳାର ସୁଲତାନ ସୁଲେମାନ ଖାନ କରରାନୀଙ୍କ ଶତ୍ରୁ ଭାବେ ସାବ୍ୟସ୍ତ କରିଥିଲେ। କିଛି ଲଢ଼େଇ ପରେ ଶେଷରେ ଓଡିଶାର ପରାଜୟ ଘଟିଥିଲ

## Indic to English Language Translation 

In [3]:
#
indic2en_model = Model(expdir='/content/model_files/indic-en')
#
INDIC = {"Assamese": "as", "Bengali": "bn", "Gujarati": "gu", "Hindi": "hi","Kannada": "kn","Malayalam": "ml", "Marathi": "mr", "Odia": "or","Punjabi": "pa","Tamil": "ta", "Telugu" : "te"}
#
languages = list(INDIC.keys())

Initializing vocab and bpe
Initializing model for translation


FileNotFoundError: ignored

In [24]:
inputs = ['एलन मस्क ने टेस्ला के 8.5 अरब डॉलर के शेयर बेचे',
 'मैं एक पेशेवर अकादमिक और शोध लेखक हूं।',
 'अमेरिका में नौकरी करो और जर्मनी में काम करो']

In [25]:
def translate2EnglishP(text, lang):
  return indic2en_model.translate_paragraph(text, INDIC[lang], 'en')
def translate2EnglishB(text, lang):
  return indic2en_model.batch_translate(text,INDIC[lang], 'en')

In [26]:
translate2EnglishB(inputs, 'Hindi')

100%|██████████| 3/3 [00:00<00:00, 2448.51it/s]


['Elon Musk sells $8.5 billion Tesla shares',
 'I am a professional academic and research writer.',
 'Work in America and work in Germany']

In [27]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [33]:
import shutil

## Testing Inference Script

In [18]:
os.curdir

'.'

In [12]:
import ast
from collections import namedtuple

import torch
from fairseq import checkpoint_utils, options, tasks, utils
from fairseq.dataclass.utils import convert_namespace_to_omegaconf
from fairseq.token_generation_constraints import pack_constraints, unpack_constraints
from fairseq_cli.generate import get_symbols_to_strip_from_output
#
from fairseq.models import register_model_architecture
from fairseq.models.transformer import base_architecture
#

import codecs
import os
import json
#
from os import truncate
from sacremoses import MosesPunctNormalizer
from sacremoses import MosesTokenizer
from sacremoses import MosesDetokenizer
from subword_nmt.apply_bpe import BPE, read_vocabulary
import codecs
from tqdm import tqdm
from indicnlp.tokenize import indic_tokenize
from indicnlp.tokenize import indic_detokenize
from indicnlp.normalize import indic_normalize
from indicnlp.transliterate import unicode_transliterate
from mosestokenizer import MosesSentenceSplitter
from indicnlp.tokenize import sentence_tokenize
#
from fairseq import checkpoint_utils, distributed_utils, options, tasks, utils
#
#Custom Interactive
#
Batch = namedtuple("Batch", "ids src_tokens src_lengths constraints")
Translation = namedtuple("Translation", "src_str hypos pos_scores alignments")


def make_batches(
    lines, cfg, task, max_positions, encode_fn, constrainted_decoding=False
):
    def encode_fn_target(x):
        return encode_fn(x)

    if constrainted_decoding:
        # Strip (tab-delimited) contraints, if present, from input lines,
        # store them in batch_constraints
        batch_constraints = [list() for _ in lines]
        for i, line in enumerate(lines):
            if "\t" in line:
                lines[i], *batch_constraints[i] = line.split("\t")

        # Convert each List[str] to List[Tensor]
        for i, constraint_list in enumerate(batch_constraints):
            batch_constraints[i] = [
                task.target_dictionary.encode_line(
                    encode_fn_target(constraint),
                    append_eos=False,
                    add_if_not_exist=False,
                )
                for constraint in constraint_list
            ]

    if constrainted_decoding:
        constraints_tensor = pack_constraints(batch_constraints)
    else:
        constraints_tensor = None

    tokens, lengths = task.get_interactive_tokens_and_lengths(lines, encode_fn)

    itr = task.get_batch_iterator(
        dataset=task.build_dataset_for_inference(
            tokens, lengths, constraints=constraints_tensor
        ),
        max_tokens=cfg.dataset.max_tokens,
        max_sentences=cfg.dataset.batch_size,
        max_positions=max_positions,
        ignore_invalid_inputs=cfg.dataset.skip_invalid_size_inputs_valid_test,
    ).next_epoch_itr(shuffle=False)
    for batch in itr:
        ids = batch["id"]
        src_tokens = batch["net_input"]["src_tokens"]
        src_lengths = batch["net_input"]["src_lengths"]
        constraints = batch.get("constraints", None)

        yield Batch(
            ids=ids,
            src_tokens=src_tokens,
            src_lengths=src_lengths,
            constraints=constraints,
        )


class Translator:
    def __init__(
        self, model_path,data_dir, checkpoint_path, batch_size=25, constrained_decoding=False
    ):

        self.constrained_decoding = constrained_decoding
        self.modl_path = model_path
        self.parser = options.get_generation_parser(interactive=True)
        # buffer_size is currently not used but we just initialize it to batch
        # size + 1 to avoid any assertion errors.
        if self.constrained_decoding:
            self.parser.set_defaults(
                path=checkpoint_path,
                remove_bpe="subword_nmt",
                num_workers=-1,
                constraints="ordered",
                batch_size=batch_size,
                buffer_size=batch_size + 1,
            )
        else:
            self.parser.set_defaults(
                path=checkpoint_path,
                remove_bpe="subword_nmt",
                num_workers=-1,
                batch_size=batch_size,
                buffer_size=batch_size + 1,
            )
        args = options.parse_args_and_arch(self.parser, input_args=[data_dir])
        # we are explictly setting src_lang and tgt_lang here
        # generally the data_dir we pass contains {split}-{src_lang}-{tgt_lang}.*.idx files from
        # which fairseq infers the src and tgt langs(if these are not passed). In deployment we dont
        # use any idx files and only store the SRC and TGT dictionaries.
        args.source_lang = "SRC"
        args.target_lang = "TGT"
        # since we are truncating sentences to max_seq_len in engine, we can set it to False here
        args.skip_invalid_size_inputs_valid_test = False

        # we have custom architechtures in this folder and we will let fairseq
        # import this
        model_file_path = os.path.join(model_path,"en-indic","model_configs")
        args.user_dir = model_file_path
        self.cfg = convert_namespace_to_omegaconf(args)

        utils.import_user_module(self.cfg.common)

        if self.cfg.interactive.buffer_size < 1:
            self.cfg.interactive.buffer_size = 1
        if self.cfg.dataset.max_tokens is None and self.cfg.dataset.batch_size is None:
            self.cfg.dataset.batch_size = 1

        assert (
            not self.cfg.generation.sampling
            or self.cfg.generation.nbest == self.cfg.generation.beam
        ), "--sampling requires --nbest to be equal to --beam"
        assert (
            not self.cfg.dataset.batch_size
            or self.cfg.dataset.batch_size <= self.cfg.interactive.buffer_size
        ), "--batch-size cannot be larger than --buffer-size"

        # Fix seed for stochastic decoding
        # if self.cfg.common.seed is not None and not self.cfg.generation.no_seed_provided:
        #     np.random.seed(self.cfg.common.seed)
        #     utils.set_torch_seed(self.cfg.common.seed)

        # if not self.constrained_decoding:
        #     self.use_cuda = torch.cuda.is_available() and not self.cfg.common.cpu
        # else:
        #     self.use_cuda = False

        self.use_cuda = torch.cuda.is_available() and not self.cfg.common.cpu

        # Setup task, e.g., translation
        self.task = tasks.setup_task(self.cfg.task)

        # Load ensemble
        overrides = ast.literal_eval(self.cfg.common_eval.model_overrides)
        self.models, self._model_args = checkpoint_utils.load_model_ensemble(
            utils.split_paths(self.cfg.common_eval.path),
            arg_overrides=overrides,
            task=self.task,
            suffix=self.cfg.checkpoint.checkpoint_suffix,
            strict=(self.cfg.checkpoint.checkpoint_shard_count == 1),
            num_shards=self.cfg.checkpoint.checkpoint_shard_count,
        )

        # Set dictionaries
        self.src_dict = self.task.source_dictionary
        self.tgt_dict = self.task.target_dictionary

        # Optimize ensemble for generation
        for model in self.models:
            if model is None:
                continue
            if self.cfg.common.fp16:
                model.half()
            if (
                self.use_cuda
                and not self.cfg.distributed_training.pipeline_model_parallel
            ):
                model.cuda()
            model.prepare_for_inference_(self.cfg)

        # Initialize generator
        self.generator = self.task.build_generator(self.models, self.cfg.generation)

        # Handle tokenization and BPE
        self.tokenizer = self.task.build_tokenizer(self.cfg.tokenizer)
        self.bpe = self.task.build_bpe(self.cfg.bpe)

        # Load alignment dictionary for unknown word replacement
        # (None if no unknown word replacement, empty if no path to align dictionary)
        self.align_dict = utils.load_align_dict(self.cfg.generation.replace_unk)

        self.max_positions = utils.resolve_max_positions(
            self.task.max_positions(), *[model.max_positions() for model in self.models]
        )

    def encode_fn(self, x):
        if self.tokenizer is not None:
            x = self.tokenizer.encode(x)
        if self.bpe is not None:
            x = self.bpe.encode(x)
        return x

    def decode_fn(self, x):
        if self.bpe is not None:
            x = self.bpe.decode(x)
        if self.tokenizer is not None:
            x = self.tokenizer.decode(x)
        return x

    def translate(self, inputs, constraints=None):
        if self.constrained_decoding and constraints is None:
            raise ValueError("Constraints cant be None in constrained decoding mode")
        if not self.constrained_decoding and constraints is not None:
            raise ValueError("Cannot pass constraints during normal translation")
        if constraints:
            constrained_decoding = True
            modified_inputs = []
            for _input, constraint in zip(inputs, constraints):
                modified_inputs.append(_input + f"\t{constraint}")
            inputs = modified_inputs
        else:
            constrained_decoding = False

        start_id = 0
        results = []
        final_translations = []
        for batch in make_batches(
            inputs,
            self.cfg,
            self.task,
            self.max_positions,
            self.encode_fn,
            constrained_decoding,
        ):
            bsz = batch.src_tokens.size(0)
            src_tokens = batch.src_tokens
            src_lengths = batch.src_lengths
            constraints = batch.constraints
            if self.use_cuda:
                src_tokens = src_tokens.cuda()
                src_lengths = src_lengths.cuda()
                if constraints is not None:
                    constraints = constraints.cuda()

            sample = {
                "net_input": {
                    "src_tokens": src_tokens,
                    "src_lengths": src_lengths,
                },
            }

            translations = self.task.inference_step(
                self.generator, self.models, sample, constraints=constraints
            )

            list_constraints = [[] for _ in range(bsz)]
            if constrained_decoding:
                list_constraints = [unpack_constraints(c) for c in constraints]
            for i, (id, hypos) in enumerate(zip(batch.ids.tolist(), translations)):
                src_tokens_i = utils.strip_pad(src_tokens[i], self.tgt_dict.pad())
                constraints = list_constraints[i]
                results.append(
                    (
                        start_id + id,
                        src_tokens_i,
                        hypos,
                        {
                            "constraints": constraints,
                        },
                    )
                )

        # sort output to match input order
        for id_, src_tokens, hypos, _ in sorted(results, key=lambda x: x[0]):
            src_str = ""
            if self.src_dict is not None:
                src_str = self.src_dict.string(
                    src_tokens, self.cfg.common_eval.post_process
                )

            # Process top predictions
            for hypo in hypos[: min(len(hypos), self.cfg.generation.nbest)]:
                hypo_tokens, hypo_str, alignment = utils.post_process_prediction(
                    hypo_tokens=hypo["tokens"].int().cpu(),
                    src_str=src_str,
                    alignment=hypo["alignment"],
                    align_dict=self.align_dict,
                    tgt_dict=self.tgt_dict,
                    remove_bpe="subword_nmt",
                    extra_symbols_to_ignore=get_symbols_to_strip_from_output(
                        self.generator
                    ),
                )
                detok_hypo_str = self.decode_fn(hypo_str)
                final_translations.append(detok_hypo_str)
        return final_translations
    #
    #engine
    INDIC = ["as", "bn", "gu", "hi", "kn", "ml", "mr", "or", "pa", "ta", "te"]


def split_sentences(paragraph, language):
    if language == "en":
        with MosesSentenceSplitter(language) as splitter:
            return splitter([paragraph])
    elif language in INDIC:
        return sentence_tokenize.sentence_split(paragraph, lang=language)


def add_token(sent, tag_infos):
    """add special tokens specified by tag_infos to each element in list
    tag_infos: list of tuples (tag_type,tag)
    each tag_info results in a token of the form: __{tag_type}__{tag}__
    """

    tokens = []
    for tag_type, tag in tag_infos:
        token = "__" + tag_type + "__" + tag + "__"
        tokens.append(token)

    return " ".join(tokens) + " " + sent


def apply_lang_tags(sents, src_lang, tgt_lang):
    tagged_sents = []
    for sent in sents:
        tagged_sent = add_token(sent.strip(), [("src", src_lang), ("tgt", tgt_lang)])
        tagged_sents.append(tagged_sent)
    return tagged_sents


def truncate_long_sentences(sents):

    MAX_SEQ_LEN = 200
    new_sents = []

    for sent in sents:
        words = sent.split()
        num_words = len(words)
        if num_words > MAX_SEQ_LEN:
            print_str = " ".join(words[:5]) + " .... " + " ".join(words[-5:])
            sent = " ".join(words[:MAX_SEQ_LEN])
            print(
                f"WARNING: Sentence {print_str} truncated to 200 tokens as it exceeds maximum length limit"
            )

        new_sents.append(sent)
    return new_sents


class Model:
    def __init__(self, expdir,model_path):
        self.expdir = expdir
        self.model_path = model_path
        self.en_tok = MosesTokenizer(lang="en")
        self.en_normalizer = MosesPunctNormalizer()
        self.en_detok = MosesDetokenizer(lang="en")
        self.xliterator = unicode_transliterate.UnicodeIndicTransliterator()
        print("Initializing vocab and bpe")
        self.vocabulary = read_vocabulary(
            codecs.open(f"{expdir}/vocab/vocab.SRC", encoding="utf-8"), 5
        )
        self.bpe = BPE(
            codecs.open(f"{expdir}/vocab/bpe_codes.32k.SRC", encoding="utf-8"),
            -1,
            "@@",
            self.vocabulary,
            None,
        )

        print("Initializing model for translation")
        # initialize the model
        self.translator = Translator(
            model_path,f"{expdir}/final_bin", f"{expdir}/model/checkpoint_best.pt", batch_size=100
        )

    # translate a batch of sentences from src_lang to tgt_lang
    def batch_translate(self, batch, src_lang, tgt_lang):

        assert isinstance(batch, list)
        preprocessed_sents = self.preprocess(batch, lang=src_lang)
        bpe_sents = self.apply_bpe(preprocessed_sents)
        tagged_sents = apply_lang_tags(bpe_sents, src_lang, tgt_lang)
        tagged_sents = truncate_long_sentences(tagged_sents)

        translations = self.translator.translate(tagged_sents)
        postprocessed_sents = self.postprocess(translations, tgt_lang)

        return postprocessed_sents

    # translate a paragraph from src_lang to tgt_lang
    def translate_paragraph(self, paragraph, src_lang, tgt_lang):

        assert isinstance(paragraph, str)
        sents = split_sentences(paragraph, src_lang)

        postprocessed_sents = self.batch_translate(sents, src_lang, tgt_lang)

        translated_paragraph = " ".join(postprocessed_sents)

        return translated_paragraph

    def preprocess_sent(self, sent, normalizer, lang):
        if lang == "en":
            return " ".join(
                self.en_tok.tokenize(
                    self.en_normalizer.normalize(sent.strip()), escape=False
                )
            )
        else:
            # line = indic_detokenize.trivial_detokenize(line.strip(), lang)
            return unicode_transliterate.UnicodeIndicTransliterator.transliterate(
                " ".join(
                    indic_tokenize.trivial_tokenize(
                        normalizer.normalize(sent.strip()), lang
                    )
                ),
                lang,
                "hi",
            ).replace(" ् ", "्")

    def preprocess(self, sents, lang):
        """
        Normalize, tokenize and script convert(for Indic)
        return number of sentences input file
        """

        if lang == "en":

            # processed_sents = Parallel(n_jobs=-1, backend="multiprocessing")(
            #     delayed(preprocess_line)(line, None, lang) for line in tqdm(sents, total=num_lines)
            # )
            processed_sents = [
                self.preprocess_sent(line, None, lang) for line in tqdm(sents)
            ]

        else:
            normfactory = indic_normalize.IndicNormalizerFactory()
            normalizer = normfactory.get_normalizer(lang)

            # processed_sents = Parallel(n_jobs=-1, backend="multiprocessing")(
            #     delayed(preprocess_line)(line, normalizer, lang) for line in tqdm(infile, total=num_lines)
            # )
            processed_sents = [
                self.preprocess_sent(line, normalizer, lang) for line in tqdm(sents)
            ]

        return processed_sents

    def postprocess(self, sents, lang, common_lang="hi"):
        """
        parse fairseq interactive output, convert script back to native Indic script (in case of Indic languages) and detokenize.
        infname: fairseq log file
        outfname: output file of translation (sentences not translated contain the dummy string 'DUMMY_OUTPUT'
        input_size: expected number of output sentences
        lang: language
        """
        postprocessed_sents = []

        if lang == "en":
            for sent in sents:
                # outfile.write(en_detok.detokenize(sent.split(" ")) + "\n")
                postprocessed_sents.append(self.en_detok.detokenize(sent.split(" ")))
        else:
            for sent in sents:
                outstr = indic_detokenize.trivial_detokenize(
                    self.xliterator.transliterate(sent, common_lang, lang), lang
                )
                # outfile.write(outstr + "\n")
                postprocessed_sents.append(outstr)
        return postprocessed_sents

    def apply_bpe(self, sents):

        return [self.bpe.process_line(sent) for sent in sents]
#
# inference script execution template
#
def preprocess_function(text_path, content_type=None):    
    with open(text_path,"r",encoding='utf-8') as f:
        data = f.read()
    #print(data)
    # pass the textand the target tanguage to be translated separated by a ";" semicolon
    #data = text_path.read().decode("utf-8")
    text = data.split(";")[0]
    target = data.split(";")[1].strip()
    print(target.strip())
    #print(data)
    return (text,target)

#
#
def predict_function(context, model):
    text,target = context
    #
    INDIC = {"Assamese": "as", "Bengali": "bn", "Gujarati": "gu", "Hindi": "hi","Kannada": "kn","Malayalam": "ml", "Marathi": "mr", "Odia": "or","Punjabi": "pa","Tamil": "ta", "Telugu" : "te"}
    #
    input_batch = split_sentences(text, language="en")
    print(input_batch)
    answer = model.batch_translate(input_batch, 'en', INDIC[target])
    return " ".join(answer)

#
def model_load_function(model_path):
    model_file_path = os.path.join(model_path, "en-indic")
    indic2en_model = Model(model_path = model_path,expdir=model_file_path)
    return indic2en_model


#
def postprocess_function(predictions, content_type=None):
    return json.dumps({"response":predictions},ensure_ascii=False, indent=4)


## Test the script

if __name__ == '__main__':
    txt_path = "/content/input.txt"
    data = preprocess_function(txt_path)
    model_path = "/content/drive/MyDrive/indic/model_files"
    path = model_load_function(model_path)
    predictions = predict_function(data,path)
    out = postprocess_function(predictions)
    print(out)
  


Odia
Initializing vocab and bpe
Initializing model for translation
['The Sun Temple was built in the 13th century and designed as a gigantic chariot of the Sun God, Surya, with twelve pairs of ornamented wheels pulled by seven horses.', 'Some of the wheels are 3 metres wide.', 'Only six of the seven horse still stand today.The temple fell into disuse after an envoy of Jahangir desecrated the temple in the early 17th century.', 'According to folklore, there was a diamond in the centre of the idol which reflected the sun rays that passed.', 'In 1627, the then Raja of Khurda took the Sun idol from Konark to the Jagannath temple in Puri.', 'The Sun temple belongs to the Kalingan school of Indian temple architecture.', 'The alignment of the Sun Temple is along the east–west direction.', 'The inner sanctum or vimana used to be surmounted by a tower or shikara but it was razed in the 19th century.', 'The audience hall or jagamohana still stands and comprises majority of the ruins.', 'The roof

100%|██████████| 16/16 [00:00<00:00, 4658.07it/s]


{
    "response": "ତ୍ରୟୋଦଶ ଶତାବ୍ଦୀରେ ନିର୍ମିତ ଏହି ସୂର୍ଯ୍ୟମନ୍ଦିର ସୂର୍ଯ୍ୟଦେବଙ୍କ ଏକ ବିଶାଳ ରଥ ଭାବେ ପରିଗଣିତ ହୋଇଥିଲା। କେତେକ ଚକା 3 ମିଟର ଚଉଡ଼ା। ସପ୍ତଦଶ ଶତାବ୍ଦୀର ପ୍ରାରମ୍ଭରେ ଜାହାଙ୍ଗୀରଙ୍କ ଦୂତ ମନ୍ଦିରକୁ ଅପବିତ୍ର କରିବା ପରେ ଏହି ମନ୍ଦିର ନଷ୍ଟ ହୋଇଯାଇଥିଲା। ଲୋକକଥା ଅନୁସାରେ ମୂର୍ତ୍ତିର ମଧ୍ୟଭାଗରେ ଗୋଟିଏ ହୀରା ଥିଲା ଯାହା ସୂର୍ଯ୍ୟକିରଣ ପ୍ରତିଫଳିତ କରୁଥିଲା। 1627 ମସିହାରେ ତତ୍କାଳୀନ ଖୋର୍ଦ୍ଧା ରାଜା ସୂର୍ଯ୍ୟମୂର୍ତ୍ତିକୁ କୋଣାର୍କରୁ ପୁରୀର ଜଗନ୍ନାଥ ମନ୍ଦିରକୁ ନେଇଯାଇଥିଲେ। ଏହି ସୂର୍ଯ୍ୟ ମନ୍ଦିର ଭାରତୀୟ ମନ୍ଦିର ସ୍ଥାପତ୍ୟର କଳିଙ୍ଗନ ବିଦ୍ୟାଳୟର ଅଟେ। ସୂର୍ଯ୍ୟମନ୍ଦିରର ପାର୍ଶ୍ୱସଜ୍ଜା ପୂର୍ବ-ପଶ୍ଚିମ ଦିଗରେ ରହିଛି। ଆଭ୍ୟନ୍ତରୀଣ ଗର୍ଭଗୃହ ବା ବିମାନ ଉପରେ ଏକ ଟାୱାର କିମ୍ବା ଶିକାରା ଥିଲା, କିନ୍ତୁ ଏହା ଉନବିଂଶ ଶତାବ୍ଦୀରେ ଧ୍ୱଂସ ହୋଇଯାଇଥିଲା। ଦର୍ଶକ ହଲ୍ ବା ଜଗମୋହନ ଏବେ ବି ଛିଡ଼ା ହୋଇ ରହିଛି ଏବଂ ସେଥିରେ ଅଧିକାଂଶ ଭଗ୍ନାବଶେଷ ରହିଛି। ନୃତ୍ୟ ଗୃହ କିମ୍ବା ନାଟମନ୍ଦିରର ଛାତ ଉଡିଯାଇଛି। ଏହା ଭଗ୍ନାବଶେଷର ପୂର୍ବ ଦିଗରେ ଏକ ଉପରକୁ ଉଠିଥିବା ପ୍ଲାଟଫର୍ମରେ ରହିଛି। ୧୫୫୯ ମସିହାରେ ମୁକୁନ୍ଦ ଗଜପତି କଟକର ସିଂହାସନ ସମ୍ଭାଳିଥିଲେ। ସେ ନିଜକୁ ଆକବରଙ୍କ ମିତ୍ର ଏବଂ ବଙ୍ଗଳାର ସୁଲତାନ ସୁଲେମାନ ଖାନ କରରାନୀଙ୍କ ଶତ୍ରୁ ଭାବେ ସାବ୍ୟସ୍ତ କରିଥିଲେ। କିଛି ଲଢ଼େଇ ପରେ ଶେଷରେ ଓ

In [20]:
%%writefile /content/input2.txt
ତ୍ରୟୋଦଶ ଶତାବ୍ଦୀରେ ନିର୍ମିତ ଏହି ସୂର୍ଯ୍ୟମନ୍ଦିର ସୂର୍ଯ୍ୟଦେବଙ୍କ ଏକ ବିଶାଳ ରଥ ଭାବେ ପରିଗଣିତ ହୋଇଥିଲା। କେତେକ ଚକା 3 ମିଟର ଚଉଡ଼ା। ସପ୍ତଦଶ ଶତାବ୍ଦୀର ପ୍ରାରମ୍ଭରେ ଜାହାଙ୍ଗୀରଙ୍କ ଦୂତ ମନ୍ଦିରକୁ ଅପବିତ୍ର କରିବା ପରେ ଏହି ମନ୍ଦିର ନଷ୍ଟ ହୋଇଯାଇଥିଲା। ଲୋକକଥା ଅନୁସାରେ ମୂର୍ତ୍ତିର ମଧ୍ୟଭାଗରେ ଗୋଟିଏ ହୀରା ଥିଲା ଯାହା ସୂର୍ଯ୍ୟକିରଣ ପ୍ରତିଫଳିତ କରୁଥିଲା। 1627 ମସିହାରେ ତତ୍କାଳୀନ ଖୋର୍ଦ୍ଧା ରାଜା ସୂର୍ଯ୍ୟମୂର୍ତ୍ତିକୁ କୋଣାର୍କରୁ ପୁରୀର ଜଗନ୍ନାଥ ମନ୍ଦିରକୁ ନେଇଯାଇଥିଲେ। ଏହି ସୂର୍ଯ୍ୟ ମନ୍ଦିର ଭାରତୀୟ ମନ୍ଦିର ସ୍ଥାପତ୍ୟର କଳିଙ୍ଗନ ବିଦ୍ୟାଳୟର ଅଟେ। ସୂର୍ଯ୍ୟମନ୍ଦିରର ପାର୍ଶ୍ୱସଜ୍ଜା ପୂର୍ବ-ପଶ୍ଚିମ ଦିଗରେ ରହିଛି। ଆଭ୍ୟନ୍ତରୀଣ ଗର୍ଭଗୃହ ବା ବିମାନ ଉପରେ ଏକ ଟାୱାର କିମ୍ବା ଶିକାରା ଥିଲା, କିନ୍ତୁ ଏହା ଉନବିଂଶ ଶତାବ୍ଦୀରେ ଧ୍ୱଂସ ହୋଇଯାଇଥିଲା। ଦର୍ଶକ ହଲ୍ ବା ଜଗମୋହନ ଏବେ ବି ଛିଡ଼ା ହୋଇ ରହିଛି ଏବଂ ସେଥିରେ ଅଧିକାଂଶ ଭଗ୍ନାବଶେଷ ରହିଛି। ନୃତ୍ୟ ଗୃହ କିମ୍ବା ନାଟମନ୍ଦିରର ଛାତ ଉଡିଯାଇଛି। ଏହା ଭଗ୍ନାବଶେଷର ପୂର୍ବ ଦିଗରେ ଏକ ଉପରକୁ ଉଠିଥିବା ପ୍ଲାଟଫର୍ମରେ ରହିଛି। ୧୫୫୯ ମସିହାରେ ମୁକୁନ୍ଦ ଗଜପତି କଟକର ସିଂହାସନ ସମ୍ଭାଳିଥିଲେ। ସେ ନିଜକୁ ଆକବରଙ୍କ ମିତ୍ର ଏବଂ ବଙ୍ଗଳାର ସୁଲତାନ ସୁଲେମାନ ଖାନ କରରାନୀଙ୍କ ଶତ୍ରୁ ଭାବେ ସାବ୍ୟସ୍ତ କରିଥିଲେ। କିଛି ଲଢ଼େଇ ପରେ ଶେଷରେ ଓଡିଶାର ପରାଜୟ ଘଟିଥିଲା। ରାଜ୍ୟର ଆଭ୍ୟନ୍ତରୀଣ ଅସ୍ଥିରତା କାରଣରୁ ମଧ୍ୟ ଏହି ହ୍ରାସ ଘଟିଛି। ୧୫୬୮ ମସିହାରେ ସୁଲତାନର ସେନାପତି କାଲାପାହାଡ଼ଙ୍କ ସେନା କୋଣାର୍କ ମନ୍ଦିରକୁ କ୍ଷତିଗ୍ରସ୍ତ କରିଥିଲେ। ବିଜୟ ସମୟରେ ଅନ୍ୟ କେତେକ ମନ୍ଦିରର କ୍ଷୟକ୍ଷତି ପାଇଁ ମଧ୍ୟ କଳାପାହାଡ଼ ଦାୟୀ ବୋଲି କୁହାଯାଏ।;Odia

Writing /content/input2.txt


## Inference script indic 2 English

In [21]:
import ast
from collections import namedtuple

import torch
from fairseq import checkpoint_utils, options, tasks, utils
from fairseq.dataclass.utils import convert_namespace_to_omegaconf
from fairseq.token_generation_constraints import pack_constraints, unpack_constraints
from fairseq_cli.generate import get_symbols_to_strip_from_output

import codecs
import os
import json
#
from os import truncate
from sacremoses import MosesPunctNormalizer
from sacremoses import MosesTokenizer
from sacremoses import MosesDetokenizer
from subword_nmt.apply_bpe import BPE, read_vocabulary
import codecs
from tqdm import tqdm
from indicnlp.tokenize import indic_tokenize
from indicnlp.tokenize import indic_detokenize
from indicnlp.normalize import indic_normalize
from indicnlp.transliterate import unicode_transliterate
from mosestokenizer import MosesSentenceSplitter
from indicnlp.tokenize import sentence_tokenize
#
from fairseq import checkpoint_utils, distributed_utils, options, tasks, utils
#
#Custom Interactive
#
Batch = namedtuple("Batch", "ids src_tokens src_lengths constraints")
Translation = namedtuple("Translation", "src_str hypos pos_scores alignments")


def make_batches(
    lines, cfg, task, max_positions, encode_fn, constrainted_decoding=False
):
    def encode_fn_target(x):
        return encode_fn(x)

    if constrainted_decoding:
        # Strip (tab-delimited) contraints, if present, from input lines,
        # store them in batch_constraints
        batch_constraints = [list() for _ in lines]
        for i, line in enumerate(lines):
            if "\t" in line:
                lines[i], *batch_constraints[i] = line.split("\t")

        # Convert each List[str] to List[Tensor]
        for i, constraint_list in enumerate(batch_constraints):
            batch_constraints[i] = [
                task.target_dictionary.encode_line(
                    encode_fn_target(constraint),
                    append_eos=False,
                    add_if_not_exist=False,
                )
                for constraint in constraint_list
            ]

    if constrainted_decoding:
        constraints_tensor = pack_constraints(batch_constraints)
    else:
        constraints_tensor = None

    tokens, lengths = task.get_interactive_tokens_and_lengths(lines, encode_fn)

    itr = task.get_batch_iterator(
        dataset=task.build_dataset_for_inference(
            tokens, lengths, constraints=constraints_tensor
        ),
        max_tokens=cfg.dataset.max_tokens,
        max_sentences=cfg.dataset.batch_size,
        max_positions=max_positions,
        ignore_invalid_inputs=cfg.dataset.skip_invalid_size_inputs_valid_test,
    ).next_epoch_itr(shuffle=False)
    for batch in itr:
        ids = batch["id"]
        src_tokens = batch["net_input"]["src_tokens"]
        src_lengths = batch["net_input"]["src_lengths"]
        constraints = batch.get("constraints", None)

        yield Batch(
            ids=ids,
            src_tokens=src_tokens,
            src_lengths=src_lengths,
            constraints=constraints,
        )


class Translator:
    def __init__(
        self, data_dir, checkpoint_path, batch_size=25, constrained_decoding=False
    ):

        self.constrained_decoding = constrained_decoding
        self.parser = options.get_generation_parser(interactive=True)
        # buffer_size is currently not used but we just initialize it to batch
        # size + 1 to avoid any assertion errors.
        if self.constrained_decoding:
            self.parser.set_defaults(
                path=checkpoint_path,
                remove_bpe="subword_nmt",
                num_workers=-1,
                constraints="ordered",
                batch_size=batch_size,
                buffer_size=batch_size + 1,
            )
        else:
            self.parser.set_defaults(
                path=checkpoint_path,
                remove_bpe="subword_nmt",
                num_workers=-1,
                batch_size=batch_size,
                buffer_size=batch_size + 1,
            )
        args = options.parse_args_and_arch(self.parser, input_args=[data_dir])
        # we are explictly setting src_lang and tgt_lang here
        # generally the data_dir we pass contains {split}-{src_lang}-{tgt_lang}.*.idx files from
        # which fairseq infers the src and tgt langs(if these are not passed). In deployment we dont
        # use any idx files and only store the SRC and TGT dictionaries.
        args.source_lang = "SRC"
        args.target_lang = "TGT"
        # since we are truncating sentences to max_seq_len in engine, we can set it to False here
        args.skip_invalid_size_inputs_valid_test = False

        # we have custom architechtures in this folder and we will let fairseq
        # import this
        args.user_dir = "model_configs"
        self.cfg = convert_namespace_to_omegaconf(args)

        utils.import_user_module(self.cfg.common)

        if self.cfg.interactive.buffer_size < 1:
            self.cfg.interactive.buffer_size = 1
        if self.cfg.dataset.max_tokens is None and self.cfg.dataset.batch_size is None:
            self.cfg.dataset.batch_size = 1

        assert (
            not self.cfg.generation.sampling
            or self.cfg.generation.nbest == self.cfg.generation.beam
        ), "--sampling requires --nbest to be equal to --beam"
        assert (
            not self.cfg.dataset.batch_size
            or self.cfg.dataset.batch_size <= self.cfg.interactive.buffer_size
        ), "--batch-size cannot be larger than --buffer-size"

        # Fix seed for stochastic decoding
        # if self.cfg.common.seed is not None and not self.cfg.generation.no_seed_provided:
        #     np.random.seed(self.cfg.common.seed)
        #     utils.set_torch_seed(self.cfg.common.seed)

        # if not self.constrained_decoding:
        #     self.use_cuda = torch.cuda.is_available() and not self.cfg.common.cpu
        # else:
        #     self.use_cuda = False

        self.use_cuda = torch.cuda.is_available() and not self.cfg.common.cpu

        # Setup task, e.g., translation
        self.task = tasks.setup_task(self.cfg.task)

        # Load ensemble
        overrides = ast.literal_eval(self.cfg.common_eval.model_overrides)
        self.models, self._model_args = checkpoint_utils.load_model_ensemble(
            utils.split_paths(self.cfg.common_eval.path),
            arg_overrides=overrides,
            task=self.task,
            suffix=self.cfg.checkpoint.checkpoint_suffix,
            strict=(self.cfg.checkpoint.checkpoint_shard_count == 1),
            num_shards=self.cfg.checkpoint.checkpoint_shard_count,
        )

        # Set dictionaries
        self.src_dict = self.task.source_dictionary
        self.tgt_dict = self.task.target_dictionary

        # Optimize ensemble for generation
        for model in self.models:
            if model is None:
                continue
            if self.cfg.common.fp16:
                model.half()
            if (
                self.use_cuda
                and not self.cfg.distributed_training.pipeline_model_parallel
            ):
                model.cuda()
            model.prepare_for_inference_(self.cfg)

        # Initialize generator
        self.generator = self.task.build_generator(self.models, self.cfg.generation)

        # Handle tokenization and BPE
        self.tokenizer = self.task.build_tokenizer(self.cfg.tokenizer)
        self.bpe = self.task.build_bpe(self.cfg.bpe)

        # Load alignment dictionary for unknown word replacement
        # (None if no unknown word replacement, empty if no path to align dictionary)
        self.align_dict = utils.load_align_dict(self.cfg.generation.replace_unk)

        self.max_positions = utils.resolve_max_positions(
            self.task.max_positions(), *[model.max_positions() for model in self.models]
        )

    def encode_fn(self, x):
        if self.tokenizer is not None:
            x = self.tokenizer.encode(x)
        if self.bpe is not None:
            x = self.bpe.encode(x)
        return x

    def decode_fn(self, x):
        if self.bpe is not None:
            x = self.bpe.decode(x)
        if self.tokenizer is not None:
            x = self.tokenizer.decode(x)
        return x

    def translate(self, inputs, constraints=None):
        if self.constrained_decoding and constraints is None:
            raise ValueError("Constraints cant be None in constrained decoding mode")
        if not self.constrained_decoding and constraints is not None:
            raise ValueError("Cannot pass constraints during normal translation")
        if constraints:
            constrained_decoding = True
            modified_inputs = []
            for _input, constraint in zip(inputs, constraints):
                modified_inputs.append(_input + f"\t{constraint}")
            inputs = modified_inputs
        else:
            constrained_decoding = False

        start_id = 0
        results = []
        final_translations = []
        for batch in make_batches(
            inputs,
            self.cfg,
            self.task,
            self.max_positions,
            self.encode_fn,
            constrained_decoding,
        ):
            bsz = batch.src_tokens.size(0)
            src_tokens = batch.src_tokens
            src_lengths = batch.src_lengths
            constraints = batch.constraints
            if self.use_cuda:
                src_tokens = src_tokens.cuda()
                src_lengths = src_lengths.cuda()
                if constraints is not None:
                    constraints = constraints.cuda()

            sample = {
                "net_input": {
                    "src_tokens": src_tokens,
                    "src_lengths": src_lengths,
                },
            }

            translations = self.task.inference_step(
                self.generator, self.models, sample, constraints=constraints
            )

            list_constraints = [[] for _ in range(bsz)]
            if constrained_decoding:
                list_constraints = [unpack_constraints(c) for c in constraints]
            for i, (id, hypos) in enumerate(zip(batch.ids.tolist(), translations)):
                src_tokens_i = utils.strip_pad(src_tokens[i], self.tgt_dict.pad())
                constraints = list_constraints[i]
                results.append(
                    (
                        start_id + id,
                        src_tokens_i,
                        hypos,
                        {
                            "constraints": constraints,
                        },
                    )
                )

        # sort output to match input order
        for id_, src_tokens, hypos, _ in sorted(results, key=lambda x: x[0]):
            src_str = ""
            if self.src_dict is not None:
                src_str = self.src_dict.string(
                    src_tokens, self.cfg.common_eval.post_process
                )

            # Process top predictions
            for hypo in hypos[: min(len(hypos), self.cfg.generation.nbest)]:
                hypo_tokens, hypo_str, alignment = utils.post_process_prediction(
                    hypo_tokens=hypo["tokens"].int().cpu(),
                    src_str=src_str,
                    alignment=hypo["alignment"],
                    align_dict=self.align_dict,
                    tgt_dict=self.tgt_dict,
                    remove_bpe="subword_nmt",
                    extra_symbols_to_ignore=get_symbols_to_strip_from_output(
                        self.generator
                    ),
                )
                detok_hypo_str = self.decode_fn(hypo_str)
                final_translations.append(detok_hypo_str)
        return final_translations
    #
    #engine
    INDIC = ["as", "bn", "gu", "hi", "kn", "ml", "mr", "or", "pa", "ta", "te"]


def split_sentences(paragraph, language):
    if language == "en":
        with MosesSentenceSplitter(language) as splitter:
            return splitter([paragraph])
    elif language in INDIC:
        return sentence_tokenize.sentence_split(paragraph, lang=language)


def add_token(sent, tag_infos):
    """add special tokens specified by tag_infos to each element in list
    tag_infos: list of tuples (tag_type,tag)
    each tag_info results in a token of the form: __{tag_type}__{tag}__
    """

    tokens = []
    for tag_type, tag in tag_infos:
        token = "__" + tag_type + "__" + tag + "__"
        tokens.append(token)

    return " ".join(tokens) + " " + sent


def apply_lang_tags(sents, src_lang, tgt_lang):
    tagged_sents = []
    for sent in sents:
        tagged_sent = add_token(sent.strip(), [("src", src_lang), ("tgt", tgt_lang)])
        tagged_sents.append(tagged_sent)
    return tagged_sents


def truncate_long_sentences(sents):

    MAX_SEQ_LEN = 200
    new_sents = []

    for sent in sents:
        words = sent.split()
        num_words = len(words)
        if num_words > MAX_SEQ_LEN:
            print_str = " ".join(words[:5]) + " .... " + " ".join(words[-5:])
            sent = " ".join(words[:MAX_SEQ_LEN])
            print(
                f"WARNING: Sentence {print_str} truncated to 200 tokens as it exceeds maximum length limit"
            )

        new_sents.append(sent)
    return new_sents


class Model:
    def __init__(self, expdir):
        self.expdir = expdir
        self.en_tok = MosesTokenizer(lang="en")
        self.en_normalizer = MosesPunctNormalizer()
        self.en_detok = MosesDetokenizer(lang="en")
        self.xliterator = unicode_transliterate.UnicodeIndicTransliterator()
        print("Initializing vocab and bpe")
        self.vocabulary = read_vocabulary(
            codecs.open(f"{expdir}/vocab/vocab.SRC", encoding="utf-8"), 5
        )
        self.bpe = BPE(
            codecs.open(f"{expdir}/vocab/bpe_codes.32k.SRC", encoding="utf-8"),
            -1,
            "@@",
            self.vocabulary,
            None,
        )

        print("Initializing model for translation")
        # initialize the model
        self.translator = Translator(
            f"{expdir}/final_bin", f"{expdir}/model/checkpoint_best.pt", batch_size=100
        )

    # translate a batch of sentences from src_lang to tgt_lang
    def batch_translate(self, batch, src_lang, tgt_lang):

        assert isinstance(batch, list)
        preprocessed_sents = self.preprocess(batch, lang=src_lang)
        bpe_sents = self.apply_bpe(preprocessed_sents)
        tagged_sents = apply_lang_tags(bpe_sents, src_lang, tgt_lang)
        tagged_sents = truncate_long_sentences(tagged_sents)

        translations = self.translator.translate(tagged_sents)
        postprocessed_sents = self.postprocess(translations, tgt_lang)

        return postprocessed_sents

    # translate a paragraph from src_lang to tgt_lang
    def translate_paragraph(self, paragraph, src_lang, tgt_lang):

        assert isinstance(paragraph, str)
        sents = split_sentences(paragraph, src_lang)

        postprocessed_sents = self.batch_translate(sents, src_lang, tgt_lang)

        translated_paragraph = " ".join(postprocessed_sents)

        return translated_paragraph

    def preprocess_sent(self, sent, normalizer, lang):
        if lang == "en":
            return " ".join(
                self.en_tok.tokenize(
                    self.en_normalizer.normalize(sent.strip()), escape=False
                )
            )
        else:
            # line = indic_detokenize.trivial_detokenize(line.strip(), lang)
            return unicode_transliterate.UnicodeIndicTransliterator.transliterate(
                " ".join(
                    indic_tokenize.trivial_tokenize(
                        normalizer.normalize(sent.strip()), lang
                    )
                ),
                lang,
                "hi",
            ).replace(" ् ", "्")

    def preprocess(self, sents, lang):
        """
        Normalize, tokenize and script convert(for Indic)
        return number of sentences input file
        """

        if lang == "en":

            # processed_sents = Parallel(n_jobs=-1, backend="multiprocessing")(
            #     delayed(preprocess_line)(line, None, lang) for line in tqdm(sents, total=num_lines)
            # )
            processed_sents = [
                self.preprocess_sent(line, None, lang) for line in tqdm(sents)
            ]

        else:
            normfactory = indic_normalize.IndicNormalizerFactory()
            normalizer = normfactory.get_normalizer(lang)

            # processed_sents = Parallel(n_jobs=-1, backend="multiprocessing")(
            #     delayed(preprocess_line)(line, normalizer, lang) for line in tqdm(infile, total=num_lines)
            # )
            processed_sents = [
                self.preprocess_sent(line, normalizer, lang) for line in tqdm(sents)
            ]

        return processed_sents

    def postprocess(self, sents, lang, common_lang="hi"):
        """
        parse fairseq interactive output, convert script back to native Indic script (in case of Indic languages) and detokenize.
        infname: fairseq log file
        outfname: output file of translation (sentences not translated contain the dummy string 'DUMMY_OUTPUT'
        input_size: expected number of output sentences
        lang: language
        """
        postprocessed_sents = []

        if lang == "en":
            for sent in sents:
                # outfile.write(en_detok.detokenize(sent.split(" ")) + "\n")
                postprocessed_sents.append(self.en_detok.detokenize(sent.split(" ")))
        else:
            for sent in sents:
                outstr = indic_detokenize.trivial_detokenize(
                    self.xliterator.transliterate(sent, common_lang, lang), lang
                )
                # outfile.write(outstr + "\n")
                postprocessed_sents.append(outstr)
        return postprocessed_sents

    def apply_bpe(self, sents):

        return [self.bpe.process_line(sent) for sent in sents]
#
# inference script execution template
#
def preprocess_function(text_path, content_type=None):    
    with open(text_path,"r",encoding='utf-8') as f:
        data = f.read()
    print(data)
    # pass the textand the target tanguage to be translated separated by a ";" semicolon
    #data = text_path.read().decode("utf-8")
    text = data.split(";")[0]
    target = data.split(";")[1].strip()
    print(target.strip())
    print(data)
    return (text,target)

#
#
def predict_function(context, model):
    text,target = context
    INDIC = {"Assamese": "as", "Bengali": "bn", "Gujarati": "gu", "Hindi": "hi","Kannada": "kn","Malayalam": "ml", "Marathi": "mr", "Odia": "or","Punjabi": "pa","Tamil": "ta", "Telugu" : "te"}
    #
    input_batch = split_sentences(text, language=INDIC[target])
    #
    print(input_batch)
    answer = model.batch_translate(input_batch, INDIC[target],'en')
    return " ".join(answer)

#
def model_load_function(model_path):
    model_file_path = os.path.join(model_path, "indic-en")
    indic2en_model = Model(expdir=model_file_path)
    return indic2en_model


#
def postprocess_function(predictions, content_type=None):
    return json.dumps({"response":predictions},ensure_ascii=False, indent=4)


## Test the script

if __name__ == '__main__':
    txt_path = "/content/input2.txt"
    data = preprocess_function(txt_path)
    model_path = "/content/model_files"
    path = model_load_function(model_path)
    predictions = predict_function(data,path)
    out = postprocess_function(predictions)
    print(out)

ତ୍ରୟୋଦଶ ଶତାବ୍ଦୀରେ ନିର୍ମିତ ଏହି ସୂର୍ଯ୍ୟମନ୍ଦିର ସୂର୍ଯ୍ୟଦେବଙ୍କ ଏକ ବିଶାଳ ରଥ ଭାବେ ପରିଗଣିତ ହୋଇଥିଲା। କେତେକ ଚକା 3 ମିଟର ଚଉଡ଼ା। ସପ୍ତଦଶ ଶତାବ୍ଦୀର ପ୍ରାରମ୍ଭରେ ଜାହାଙ୍ଗୀରଙ୍କ ଦୂତ ମନ୍ଦିରକୁ ଅପବିତ୍ର କରିବା ପରେ ଏହି ମନ୍ଦିର ନଷ୍ଟ ହୋଇଯାଇଥିଲା। ଲୋକକଥା ଅନୁସାରେ ମୂର୍ତ୍ତିର ମଧ୍ୟଭାଗରେ ଗୋଟିଏ ହୀରା ଥିଲା ଯାହା ସୂର୍ଯ୍ୟକିରଣ ପ୍ରତିଫଳିତ କରୁଥିଲା। 1627 ମସିହାରେ ତତ୍କାଳୀନ ଖୋର୍ଦ୍ଧା ରାଜା ସୂର୍ଯ୍ୟମୂର୍ତ୍ତିକୁ କୋଣାର୍କରୁ ପୁରୀର ଜଗନ୍ନାଥ ମନ୍ଦିରକୁ ନେଇଯାଇଥିଲେ। ଏହି ସୂର୍ଯ୍ୟ ମନ୍ଦିର ଭାରତୀୟ ମନ୍ଦିର ସ୍ଥାପତ୍ୟର କଳିଙ୍ଗନ ବିଦ୍ୟାଳୟର ଅଟେ। ସୂର୍ଯ୍ୟମନ୍ଦିରର ପାର୍ଶ୍ୱସଜ୍ଜା ପୂର୍ବ-ପଶ୍ଚିମ ଦିଗରେ ରହିଛି। ଆଭ୍ୟନ୍ତରୀଣ ଗର୍ଭଗୃହ ବା ବିମାନ ଉପରେ ଏକ ଟାୱାର କିମ୍ବା ଶିକାରା ଥିଲା, କିନ୍ତୁ ଏହା ଉନବିଂଶ ଶତାବ୍ଦୀରେ ଧ୍ୱଂସ ହୋଇଯାଇଥିଲା। ଦର୍ଶକ ହଲ୍ ବା ଜଗମୋହନ ଏବେ ବି ଛିଡ଼ା ହୋଇ ରହିଛି ଏବଂ ସେଥିରେ ଅଧିକାଂଶ ଭଗ୍ନାବଶେଷ ରହିଛି। ନୃତ୍ୟ ଗୃହ କିମ୍ବା ନାଟମନ୍ଦିରର ଛାତ ଉଡିଯାଇଛି। ଏହା ଭଗ୍ନାବଶେଷର ପୂର୍ବ ଦିଗରେ ଏକ ଉପରକୁ ଉଠିଥିବା ପ୍ଲାଟଫର୍ମରେ ରହିଛି। ୧୫୫୯ ମସିହାରେ ମୁକୁନ୍ଦ ଗଜପତି କଟକର ସିଂହାସନ ସମ୍ଭାଳିଥିଲେ। ସେ ନିଜକୁ ଆକବରଙ୍କ ମିତ୍ର ଏବଂ ବଙ୍ଗଳାର ସୁଲତାନ ସୁଲେମାନ ଖାନ କରରାନୀଙ୍କ ଶତ୍ରୁ ଭାବେ ସାବ୍ୟସ୍ତ କରିଥିଲେ। କିଛି ଲଢ଼େଇ ପରେ ଶେଷରେ ଓଡିଶାର ପରାଜୟ ଘଟିଥିଲା

FileNotFoundError: ignored